## Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

In [9]:
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

1. Загрузите датасет. Описание датасета можно посмотреть здесь: http://www.dt.fee.unicamp.br/~tiago/smsspamcollection

In [1]:
with open('smsspamcollection/SMSSpamCollection.txt', 'r') as f:
    sms = f.read().splitlines()

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [5]:
sms[0]

'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [7]:
sms[5]

"spam\tFreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"

3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [13]:
# label - метки: 'ham', 'spam'
# text - текст
label, text = zip(*[line.split('\t') for line in sms])

# label_bi - метки класса: 0 for ham, 1 for spam
label_bi = np.array([0 if t=='ham' else 1 for t in label])

In [14]:
label[0:5]

('ham', 'ham', 'spam', 'ham', 'ham')

In [15]:
text[0:5]

('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though")

In [16]:
label_bi[0:5]

array([0, 0, 1, 0, 0])

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [17]:
# CountVectorizer for text
count_vect = CountVectorizer()
X = count_vect.fit_transform(sms_text)
X.shape

(5574, 8713)

5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [22]:
pipeline = Pipeline([('count_vect', CountVectorizer()),
                     ('log_reg', LogisticRegression(random_state=2))
                    ])

In [23]:
# cross_val_score for f1 metric for log_reg
score = cross_val_score(pipeline, text, label_bi, scoring='f1', cv=10).mean()
print('Cross_val_score f1: %.4f' % score)

Cross_val_score f1: 0.9311


In [20]:
with open('ans1.txt', 'w') as f:
    ans1 = str(round(score, 1))
    f.write(ans1)

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [24]:
test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
            "FreeMsg: Txt: claim your reward of 3 hours talk time",
            "Have you visited the last lecture on physics?",
            "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
            "Only 99$"]

In [25]:
# fitting full dataset
pipeline.fit(text, label_bi)

Pipeline(steps=[('count_vect', CountVectorizer()),
                ('log_reg', LogisticRegression(random_state=2))])

In [26]:
# predicting labels for test
test_pr = pipeline.predict(test)
test_pr

array([1, 1, 0, 0, 0])

In [28]:
with open('ans2.txt', 'w') as f:
    ans2 = ' '.join(map(str, test_pr))
    f.write(ans2)

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [37]:
# testing log_reg the model in different ngram ranges
ngram_range = [(2,2), (3,3), (1,3)]

scores = []
for ngram in ngram_range:
    pipeline = Pipeline([('count_vect', CountVectorizer(ngram_range=ngram)), ('log_reg', LogisticRegression())])
    score = cross_val_score(pipeline, text, label_bi, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Ngram_range: ', ngram)
    print('Cross-val-score f1: %.4f\n' % score)



Ngram_range:  (2, 2)
Cross-val-score f1: 0.8168

Ngram_range:  (3, 3)
Cross-val-score f1: 0.7250

Ngram_range:  (1, 3)
Cross-val-score f1: 0.9223



In [38]:
with open('ans3.txt', 'w') as f:
    ans3 = ' '.join(map(str, ['%.2f' % i for i in scores]))
    f.write(ans3)

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

 По какой-то причине  обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно  посчитайте частоты слов и обучите классификатор.  

In [39]:
# testing mult_nb the model in different ngram ranges
scores = []
for ngram in ngram_range:
    X_counts = CountVectorizer(ngram_range=ngram).fit_transform(text)
    mn_nb = MultinomialNB()
    score = cross_val_score(mn_nb, X_counts, label_bi, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Ngram_range: ', ngram)
    print('Cross-val-score f1: %.4f\n' % score)

Ngram_range:  (2, 2)
Cross-val-score f1: 0.6451

Ngram_range:  (3, 3)
Cross-val-score f1: 0.3786

Ngram_range:  (1, 3)
Cross-val-score f1: 0.8878



In [40]:
with open('ans4.txt', 'w') as f:
    ans4 = ' '.join(map(str, ['%.2f' % i for i in scores]))
    f.write(ans4)


9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [35]:
# cross_val_score for f1 metric for log_reg with tf_idf
pipeline = Pipeline([('tf_idf', TfidfVectorizer()),
                     ('log_reg', LogisticRegression())])
score = cross_val_score(pipeline, text, label_bi, scoring='f1', cv=10).mean()
print('Cross-val-score f1: %.4f\n' % score)

Cross-val-score f1: 0.8784



In [36]:
with open('ans5.txt', 'w') as f:
    ans5 = str(-1)
    f.write(ans5)